# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Project Setup

In [19]:
# all import statements needed for the project, for example:

import os
import re
import requests
import bs4
from bs4 import BeautifulSoup

import math
from math import radians, sin, cos, sqrt, atan2

import sqlite3
import sqlalchemy as db

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

import folium
from folium.plugins import HeatMap

In [20]:
# any constants you might need; some have been added for you, and 
# some you need to fill in

TLC_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_ZONES_DIR = ""
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
WEATHER_CSV_DIR = ""

CRS = 4326  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [21]:
# Make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise

## Part 1: Data Preprocessing

### Load Taxi Zones

In [22]:
def load_taxi_zones():
    """
    Load and transform NYC taxi zone shapefile data to WGS84 coordinates.

    Returns:
        gpd.GeoDataFrame: GeoDataFrame containing taxi zone data with columns:
            - LocationID: unique identifier for each zone
            - zone: name of the taxi zone
            - borough: borough the zone is in
            - geometry: geometric boundaries of the zone

    Raises:
        ValueError: If the taxi zones shapefile cannot be loaded
    """
    try:
        # use WGS84 cord
        taxi_zones = gpd.read_file('taxi_zones.shp').to_crs(CRS)
        return taxi_zones[['LocationID', 'zone', 'borough', 'geometry']]
    except Exception as e:
        print(f"Error loading shapefile: {e}")
        raise ValueError("Could not load taxi zones shapefile")

In [23]:
def lookup_coords_for_taxi_zone_id(
    zone_loc_id: int, 
    loaded_taxi_zones: gpd.GeoDataFrame
):
    r"""
    Look up centroid coordinates for a given taxi zone ID.

    Args:
        zone_loc_id: LocationID of the taxi zone to find coordinates for
        loaded_taxi_zones: GeoDataFrame containing taxi zone data

    Returns:
        tuple: (latitude, longitude) coordinates of zone centroid if found
        None: if zone lookup fails

    Raises:
        ValueError: If no taxi zone found for the given LocationID
    """
    try:
        zone = loaded_taxi_zones[loaded_taxi_zones['LocationID'] == zone_loc_id]
        if zone.empty:
            raise ValueError(f"No taxi zone found for LocationID: {zone_loc_id}")
        
        centroid = zone.geometry.centroid.iloc[0]
        return (centroid.y, centroid.x)
        
    except ValueError as e:
        print(f"Value error: {e}")
        return None
        
    except Exception as e:
        print(f"Unexpected error looking up coordinates: {e}")
        return None

### Calculate Sample Size

In [24]:
def calculate_sample_size(
   population: int,
   confidence_level: float = 0.95,
   margin_of_error: float = 0.05,
   p: float = 0.5
) -> int:
   r"""
   Calculate required sample size using Cochran's formula.

   Args:
       population: Total size of the population to sample from
       confidence_level: Desired confidence level (default 0.95)
       margin_of_error: Acceptable margin of error (default 0.05)
       p: Population proportion (default 0.5)

   Returns:
       int: Required sample size, rounded up to nearest integer

   Notes:
       Uses Cochran's formula for calculating sample size:
       n = (z^2 * p * (1-p)) / e^2
       where:
       z = z-score for confidence level
       e = margin of error
       p = population proportion
   """
   # Z-score lookup for common confidence levels
   z = {0.90: 1.645, 0.95: 1.96, 0.99: 2.576}[confidence_level]
   e = margin_of_error

   # Calculate base sample size
   numerator = (z**2) * p * (1 - p)
   denominator = e**2
   sample_size = numerator / denominator

   # Adjust for finite population if provided
   if population:
       sample_size = (sample_size * population) / (sample_size + population - 1)

   return math.ceil(sample_size)

### Common Functions

In [25]:
def get_all_urls_from_tlc_page(tlc_url: str) -> list[str]:
   r"""
   Extract all URLs from the TLC (Taxi & Limousine Commission) webpage.

   Args:
       tlc_url: URL of the TLC webpage to scrape

   Returns:
       list[str]: List of all URLs found on the page

   Raises:
       RequestException: If webpage cannot be accessed
       ParserError: If HTML parsing fails
   """
   response = requests.get(tlc_url)
   soup = BeautifulSoup(response.content, 'html.parser')
   
   # Find all links and extract URLs
   links = soup.find_all('a', href=True)
   urls = [link['href'] for link in links]
   
   return urls

In [26]:
def find_taxi_parquet_urls(all_urls: list[str]) -> list[str]:
   r"""
   Find URLs of yellow taxi parquet files from a list of URLs.
   
   Matches parquet files for yellow taxis from 2020-2024 with valid month numbers.

   Args:
       all_urls: List of URLs to search through

   Returns:
       list[str]: List of matched yellow taxi parquet file URLs

   Note:
       Expected to find 57 parquet files for the period 2020-2024
   """
   # Define regex pattern for yellow taxi parquet files (2020-2024)
   yellow_taxi_pattern = re.compile(
       r'.*yellow_trip[-]?data_202[0-4]-(0[1-9]|1[0-2])\.parquet$',
       re.IGNORECASE
   )
   
   # Filter URLs matching the pattern
   yellow_taxi_links = [
       url.strip() for url in all_urls 
       if yellow_taxi_pattern.match(url.strip())
   ]
   
   # Log results
   print(f"Found {len(yellow_taxi_links)} yellow taxi parquet files")  # should be 57
   if yellow_taxi_links:
       print("Sample URL:", yellow_taxi_links[0])
       
   return yellow_taxi_links

In [27]:
def find_uber_parquet_urls(all_urls: list[str]) -> list[str]:
   r"""
   Find URLs of Uber (FHVHV) parquet files from a list of URLs.
   
   Matches parquet files for Uber trips from 2020-2024 with valid month numbers.

   Args:
       all_urls: List of URLs to search through

   Returns:
       list[str]: List of matched Uber parquet file URLs

   Note:
       Expected to find 57 parquet files for the period 2020-2024
   """
   # Define regex pattern for Uber parquet files (2020-2024)
   uber_pattern = re.compile(
       r'.*fhvhv_trip[-]?data_202[0-4]-(0[1-9]|1[0-2])\.parquet$',
       re.IGNORECASE
   )
   
   # Filter URLs matching the pattern
   uber_links = [
       url.strip() for url in all_urls 
       if uber_pattern.match(url.strip())
   ]
   
   # Log results
   print(f"Found {len(uber_links)} fhvhv parquet files")  # should be 57
   if uber_links:
       print("Sample URL:", uber_links[0])
       
   return uber_links

### Process Taxi Data

In [28]:
def get_and_clean_taxi_month(url: str) -> pd.DataFrame | None:
   r"""
   Download, sample, and clean monthly yellow taxi trip data.

   Args:
       url: URL or local path to parquet file containing taxi data

   Returns:
       pd.DataFrame: Cleaned and sampled taxi data with standardized columns
       None: If processing fails

   Notes:
       - Downloads data if not cached locally
       - Samples data using Cochran's formula
       - Standardizes column names and formats
       - Adds geolocation coordinates
       - Identifies airport trips
       - Fills missing values
   """
   try:
       # Load or download parquet file
       filename = url.split('/')[-1]
       if os.path.exists(f"data/{filename}"):
           taxi_df = pd.read_parquet(f"data/{filename}")
       else:
           taxi_df = pd.read_parquet(url)
           os.makedirs("data", exist_ok=True)
           taxi_df.to_parquet(f"data/{filename}")
       
       # Sample data
       population_size = len(taxi_df)
       sample_size = calculate_sample_size(population_size)
       taxi_df = taxi_df.sample(n=sample_size, random_state=42)
       
       # Define required and optional columns
       required_columns = ['tpep_pickup_datetime']
       optional_columns = [
           'trip_distance', 'extra', 'mta_tax', 'tip_amount', 
           'tolls_amount', 'improvement_surcharge', 'total_amount',
           'congestion_surcharge', 'Airport_fee', 'PULocationID', 
           'DOLocationID', 'RatecodeID', 'tpep_dropoff_datetime'
       ]
       
       # Validate and select columns
       missing_columns = [col for col in required_columns if col not in taxi_df.columns]
       if missing_columns:
           raise ValueError(f"Missing required columns: {missing_columns}")
       
       available_columns = required_columns + [col for col in optional_columns if col in taxi_df.columns]
       taxi_df = taxi_df[available_columns]
       
       # Add coordinates from taxi zones
       loaded_taxi_zones = load_taxi_zones()
       taxi_df['pickup_coords'] = taxi_df['PULocationID'].apply(
           lambda loc_id: lookup_coords_for_taxi_zone_id(loc_id, loaded_taxi_zones)
       )
       taxi_df['dropoff_coords'] = taxi_df['DOLocationID'].apply(
           lambda loc_id: lookup_coords_for_taxi_zone_id(loc_id, loaded_taxi_zones)
       )
       taxi_df = taxi_df.dropna(subset=['pickup_coords', 'dropoff_coords'])
       
       # Process datetime fields
       taxi_df['tpep_pickup_datetime'] = pd.to_datetime(taxi_df['tpep_pickup_datetime'])
       taxi_df['tpep_dropoff_datetime'] = pd.to_datetime(taxi_df['tpep_dropoff_datetime'])
       taxi_df["weekday_num"] = taxi_df["tpep_dropoff_datetime"].dt.weekday + 1
       
       # Calculate total amount if missing
       taxi_df['total_amount'] = taxi_df.apply(
           lambda row: (
               row['extra'] + row['fare_amount'] + row['mta_tax'] + 
               row['airport_fee'] + row['Improvement_surcharge'] + 
               row['tolls_amount'] + row['congestion_surcharge']
           ) if pd.isna(row['total_amount']) and 
                row[['fare_amount']].notna().all()
           else row['total_amount'],
           axis=1
       )
       
       # Identify airport trips
       taxi_df['airport'] = 'not airport'
       taxi_df.loc[taxi_df['RatecodeID'] == 2, 'airport'] = 'JFK'
       taxi_df.loc[taxi_df['RatecodeID'] == 3, 'airport'] = 'EWR'
       taxi_df.loc[
           (taxi_df['Airport_fee'] == 1.75) & (taxi_df['RatecodeID'] != 2),
           'airport'
       ] = 'LGA'

       # Clean and standardize
       taxi_df = taxi_df.drop(columns=['PULocationID', 'DOLocationID'])
       taxi_df = taxi_df.rename(columns={
           'tpep_pickup_datetime': 'pickup_datetime', 
           'tpep_dropoff_datetime': 'dropoff_datetime'
       })
       
       # Fill missing values
       columns_to_fill = [
           'trip_distance', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 
           'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'Airport_fee'
       ]
       taxi_df[columns_to_fill] = taxi_df[columns_to_fill].fillna(0)
       
       # Format coordinates
       taxi_df['pickup_coords'] = taxi_df['pickup_coords'].apply(lambda x: f"{x[0]},{x[1]}")
       taxi_df['dropoff_coords'] = taxi_df['dropoff_coords'].apply(lambda x: f"{x[0]},{x[1]}")
       
       return taxi_df
       
   except Exception as e:
       print(f"Error processing {url}: {e}")
       return None

In [29]:
def get_and_clean_taxi_data(parquet_urls):
    all_taxi_dataframes = []
    
    for parquet_url in parquet_urls:
        taxi_df = get_and_clean_taxi_month(parquet_url)
        if taxi_df is not None:
            all_taxi_dataframes.append(taxi_df)
    
    if not all_taxi_dataframes:
        raise ValueError("No valid taxi data found")
        
    taxi_data = pd.concat(all_taxi_dataframes)
    return taxi_data

In [30]:
def get_taxi_data():
    all_urls = get_all_urls_from_tlc_page(TLC_URL)
    all_parquet_urls = find_taxi_parquet_urls(all_urls)
    taxi_data = get_and_clean_taxi_data(all_parquet_urls)
    return taxi_data

In [31]:
taxi_data = get_taxi_data()

Found 57 yellow taxi parquet files
Sample URL: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet
Error loading shapefile: taxi_zones.shp: No such file or directory
Error processing https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet: Could not load taxi zones shapefile
Error loading shapefile: taxi_zones.shp: No such file or directory
Error processing https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet: Could not load taxi zones shapefile
Error loading shapefile: taxi_zones.shp: No such file or directory
Error processing https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-03.parquet: Could not load taxi zones shapefile
Error loading shapefile: taxi_zones.shp: No such file or directory
Error processing https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-04.parquet: Could not load taxi zones shapefile
Error loading shapefile: taxi_zones.shp: No such file or directory
Er

ValueError: No valid taxi data found

In [ ]:
taxi_data.head()

In [ ]:
taxi_data.info()

In [ ]:
taxi_data.describe()

### Processing Uber Data

In [ ]:
def get_and_clean_uber_month(url: str) -> pd.DataFrame | None:
   r"""
   Download, sample and clean monthly Uber (FHVHV) trip data.

   Args:
       url: URL or local path to parquet file containing Uber data

   Returns:
       pd.DataFrame: Cleaned and sampled Uber data with standardized columns 
       None: If processing fails

   Notes:
       - Downloads data if not cached locally
       - Samples data using Cochran's formula
       - Filters for Uber trips only (HV0003)
       - Adds geolocation coordinates and airport identification
       - Standardizes column names and formats
       - Fills missing values
   """
   try:
       # Load or download parquet file
       filename = url.split('/')[-1]
       if os.path.exists(f"data/{filename}"):
           uber_df = pd.read_parquet(f"data/{filename}")
       else:
           uber_df = pd.read_parquet(url)
           os.makedirs("data", exist_ok=True)
           uber_df.to_parquet(f"data/{filename}")

       # Sample data
       population_size = len(uber_df)
       sample_size = calculate_sample_size(population_size)
       uber_df = uber_df.sample(n=sample_size, random_state=42)

       # Define required and optional columns
       required_columns = [
           'hvfhs_license_num', 'pickup_datetime'
       ]
       optional_columns = [
           'trip_miles', 'base_passenger_fare', 'tolls', 'sales_tax', 
           'congestion_surcharge', 'airport_fee', 'driver_pay', 'bcf',
           'PULocationID', 'DOLocationID', 'dropoff_datetime', 'tips'
       ]

       # Validate columns
       if not all(col in uber_df.columns for col in required_columns):
           raise ValueError(
               f"Missing required columns: {[col for col in required_columns if col not in uber_df.columns]}"
           )

       # Select columns and filter for Uber trips
       available_columns = required_columns + [col for col in optional_columns if col in uber_df.columns]
       uber_df = uber_df[available_columns]
       uber_df = uber_df[uber_df['hvfhs_license_num'] == 'HV0003']

       # Add coordinates from taxi zones
       loaded_taxi_zones = load_taxi_zones()
       uber_df['pickup_coords'] = uber_df['PULocationID'].apply(
           lambda loc_id: lookup_coords_for_taxi_zone_id(loc_id, loaded_taxi_zones)
       )
       uber_df['dropoff_coords'] = uber_df['DOLocationID'].apply(
           lambda loc_id: lookup_coords_for_taxi_zone_id(loc_id, loaded_taxi_zones)
       )
       uber_df = uber_df.dropna(subset=['pickup_coords', 'dropoff_coords'])

       # Process datetime fields
       uber_df['pickup_datetime'] = pd.to_datetime(uber_df['pickup_datetime'])
       uber_df['dropoff_datetime'] = pd.to_datetime(uber_df['dropoff_datetime']) 
       uber_df["weekday_num"] = uber_df["dropoff_datetime"].dt.weekday + 1

       # Calculate total amount
       uber_df['total_amount'] = uber_df.apply(
           lambda row: (
               row['base_passenger_fare'] + row['tolls'] + row['sales_tax'] +
               row['airport_fee'] + row['congestion_surcharge'] +
               row['driver_pay'] + row['bcf']
           ),
           axis=1
       )

       # Define haversine distance calculation
       def haversine(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
           """Calculate haversine distance between two points."""
           R = 6371
           lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
           dlat = lat2 - lat1
           dlon = lon2 - lon1
           a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
           c = 2 * atan2(sqrt(a), sqrt(1-a))
           return R * c

       # Define airport locations
       airports = {
           "JFK": {"lat": 40.6413, "lon": -73.7781, "radius": 5},
           "LGA": {"lat": 40.7769, "lon": -73.8740, "radius": 5},
           "EWR": {"lat": 40.6895, "lon": -74.1745, "radius": 5}
       }

       def assign_airport(row: pd.Series) -> str:
           """Determine if trip starts or ends at an airport."""
           pickup_coords = row['pickup_coords']
           dropoff_coords = row['dropoff_coords']
           pickup_lat, pickup_lon = pickup_coords
           dropoff_lat, dropoff_lon = dropoff_coords

           for airport, info in airports.items():
               pickup_distance = haversine(pickup_lat, pickup_lon, info['lat'], info['lon'])
               if pickup_distance <= info['radius']:
                   return airport

               dropoff_distance = haversine(dropoff_lat, dropoff_lon, info['lat'], info['lon'])
               if dropoff_distance <= info['radius']:
                   return airport

           return "not airport"

       # Identify airport trips
       uber_df['airport'] = uber_df.apply(assign_airport, axis=1)

       # Clean and standardize
       uber_df = uber_df.drop(columns=['PULocationID', 'DOLocationID'])
       columns_to_fill = [
           'trip_miles', 'base_passenger_fare', 'tolls', 'sales_tax',
           'congestion_surcharge', 'airport_fee', 'driver_pay', 'bcf'
       ]
       uber_df[columns_to_fill] = uber_df[columns_to_fill].fillna(0)
       
       # Format coordinates as strings
       uber_df['pickup_coords'] = uber_df['pickup_coords'].apply(lambda x: f"{x[0]},{x[1]}")
       uber_df['dropoff_coords'] = uber_df['dropoff_coords'].apply(lambda x: f"{x[0]},{x[1]}")

       return uber_df

   except Exception as e:
       print(f"Error processing {url}: {e}")
       return None

In [ ]:
def get_and_clean_uber_data(parquet_urls):
    all_uber_dataframes = []
    
    for parquet_url in parquet_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_uber_month(parquet_url)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_uber_dataframes.append(dataframe)

    # create one gigantic dataframe with data from every month needed
    uber_data = pd.concat(all_uber_dataframes)
    return uber_data

In [ ]:
def get_uber_data():
    all_urls = get_all_urls_from_tlc_page(TLC_URL)
    all_parquet_urls = find_uber_parquet_urls(all_urls)
    taxi_data = get_and_clean_uber_data(all_parquet_urls)
    return taxi_data

In [ ]:
uber_data = get_uber_data()

In [ ]:
uber_data.head()

In [ ]:
uber_data.info()

In [ ]:
uber_data.describe()

### Processing Weather Data

In [ ]:
def get_all_weather_csvs(directory: str | None = None) -> list[str]:
   r"""
   Get URLs for weather data CSV files from 2020-2024.

   Args:
       directory: Optional local directory path (not used currently)

   Returns:
       list[str]: List of URLs for weather data CSV files

   Notes:
       Currently returns hardcoded GitHub URLs for 2020-2024 weather data
   """
   weather_urls = [
       "https://raw.githubusercontent.com/Joanna-Wu-Weijia/4501-Final-Project/refs/heads/main/weather%20data/2020_weather.csv",
       "https://raw.githubusercontent.com/Joanna-Wu-Weijia/4501-Final-Project/refs/heads/main/weather%20data/2021_weather.csv", 
       "https://raw.githubusercontent.com/Joanna-Wu-Weijia/4501-Final-Project/refs/heads/main/weather%20data/2022_weather.csv",
       "https://raw.githubusercontent.com/Joanna-Wu-Weijia/4501-Final-Project/refs/heads/main/weather%20data/2023_weather.csv",
       "https://raw.githubusercontent.com/Joanna-Wu-Weijia/4501-Final-Project/refs/heads/main/weather%20data/2024_weather.csv",
   ]
   return weather_urls

In [ ]:
def clean_month_weather_data_hourly(csv_file: str) -> pd.DataFrame:
   r"""
   Clean and process hourly weather data from CSV file.

   Args:
       csv_file: Path to weather data CSV file

   Returns:
       pd.DataFrame: Cleaned weather data with standardized columns and formats

   Notes:
       - Standardizes column names
       - Maps weather type codes to human-readable labels
       - Converts data types and handles missing values
       - Adds derived columns for hour, weekday, and severe weather
   """
   # Load and select columns
   weather_data = pd.read_csv(csv_file)
   weather_data = weather_data[[
       'DATE', 'HourlyPresentWeatherType', 'HourlyDryBulbTemperature',
       'HourlyPrecipitation', 'HourlyWindSpeed'
   ]]

   # Standardize column names
   weather_data = weather_data.rename(columns={
       'DATE': 'date',
       'HourlyPresentWeatherType': 'hourly weather type',
       'HourlyDryBulbTemperature': 'hourly temperature',
       'HourlyPrecipitation': 'hourly precipitation',
       'HourlyWindSpeed': 'hourly windspeed'
   })

   # Convert numeric columns
   numeric_columns = ['hourly temperature', 'hourly precipitation', 'hourly windspeed']
   for col in numeric_columns:
       weather_data[col] = pd.to_numeric(weather_data[col], errors='coerce')

   # Define weather type mapping
   weather_mapping = {
       '-RA:02 |RA |RA': 'rain',
       '-RA:02 BR:1 |RA |RA': 'rain/mist',
       'BR:1 ||': 'mist',
       'HZ:7 |FU |HZ': 'haze/smoke',
       'RA:02 BR:1 |RA |RA': 'rain/mist',
       'FG:2 |FG |': 'fog',
       '-SN:03 |SN |': 'snow',
       '-SN:03 BR:1 |SN |': 'snow/mist',
       '+RA:02 |RA |RA': 'rain',
       '|SN |': 'snow',
       '+SN:03 |SN s |': 'heavy snow',
       '+SN:03 FZ:8 FG:2 |FG SN |': 'snow/frezzing/fog',
       '-SN:03 FZ:8 FG:2 |FG SN |': 'snow/frezzing/fog',
       'SN:03 FZ:8 FG:2 |FG SN |': 'snow/frezzing/fog',
       '+RA:02 FG:2 |FG RA |RA': 'rain/fog',
       'HZ:7 ||HZ': 'haze',
       '|RA |': 'rain',
       'RA:02 |RA |RA': 'rain',
       'UP:09 ||': 'unknown',
       'UP:09 BR:1 ||': 'mist',
       '+RA:02 BR:1 |RA |RA': 'rain',
       '-RA:02 ||': 'rain',
       'RA:02 FG:2 |FG RA |RA': 'rain/fog',
       '-RA:02 FG:2 |FG RA |RA': 'rain/fog',
       'SN:03 |SN s |s': 'snow',
       '-SN:03 FG:2 |FG SN |': 'snow/fog',
       'SN:03 FG:2 |FG SN |': 'snow/fog'
   }

   # Map weather types and handle special cases
   weather_data['hourly weather type'] = weather_data['hourly weather type'].map(weather_mapping)
   weather_data.loc[weather_data['hourly precipitation'] == 0, 'hourly weather type'] = 'sunny'
   weather_data.loc[
       (weather_data['hourly precipitation'].isna()) & 
       (weather_data['hourly weather type'].isna()),
       'hourly weather type'
   ] = 'unknown'

   # Process datetime and add derived columns
   weather_data['date'] = pd.to_datetime(weather_data['date'])
   weather_data['hour'] = weather_data['date'].dt.hour
   weather_data['weekday_num'] = weather_data['date'].dt.weekday

   # Define and mark severe weather conditions
   severe_weather_conditions = [
       '-SN:03 |SN |', '-SN:03 BR:1 |SN |', '+RA:02 |RA |RA', '|SN |',
       '+SN:03 |SN s |', '+SN:03 FZ:8 FG:2 |FG SN |', '-SN:03 FZ:8 FG:2 |FG SN |',
       'SN:03 FZ:8 FG:2 |FG SN |', '+RA:02 FG:2 |FG RA |RA', '+RA:02 BR:1 |RA |RA',
       'SN:03 |SN s |s', '-SN:03 FG:2 |FG SN |', 'SN:03 FG:2 |FG SN |'
   ]

   weather_data['severe weather'] = 0
   weather_data.loc[weather_data['hourly weather type'].isin(
       [weather_mapping[condition] for condition in severe_weather_conditions]
   ), 'severe weather'] = 1
   weather_data.loc[weather_data['hourly weather type'] == 'unknown', 'severe weather'] = None

   return weather_data

In [ ]:
def clean_month_weather_data_daily(csv_file: str) -> pd.DataFrame:
   r"""
   Clean and aggregate hourly weather data to daily summaries.

   Args:
       csv_file: Path to weather data CSV file

   Returns:
       pd.DataFrame: Daily aggregated weather data with columns:
           - date: Date of weather records
           - daily weather type: Dominant weather type for the day
           - daily temperature: Average temperature
           - daily precipitation: Average precipitation
           - daily windspeed: Average wind speed

   Notes:
       - Aggregates hourly data to daily summaries
       - Determines daily weather type based on priority (snow > rain > other)
       - Averages numeric measurements (temperature, precipitation, wind speed)
   """
   # Load and select columns
   weather_data = pd.read_csv(csv_file)
   weather_data = weather_data[[
       'DATE', 'HourlyPresentWeatherType', 'HourlyDryBulbTemperature',
       'HourlyPrecipitation', 'HourlyWindSpeed'
   ]]
   
   # Standardize column names
   weather_data.columns = [
       'datetime', 'weather_type', 'temperature', 
       'precipitation', 'wind_speed'
   ]

   # Convert data types
   weather_data['date'] = pd.to_datetime(weather_data['datetime']).dt.date
   numeric_columns = ['temperature', 'precipitation', 'wind_speed']
   for col in numeric_columns:
       weather_data[col] = pd.to_numeric(weather_data[col], errors='coerce')

   # Define weather type mapping
   weather_mapping = {
       # Rain conditions
       '-RA:02 |RA |RA': 'rain',
       '-RA:02 BR:1 |RA |RA': 'rain', 
       'RA:02 BR:1 |RA |RA': 'rain',
       '+RA:02 |RA |RA': 'rain',
       '+RA:02 FG:2 |FG RA |RA': 'rain',
       '|RA |': 'rain',
       'RA:02 |RA |RA': 'rain',
       '+RA:02 BR:1 |RA |RA': 'rain',
       '-RA:02 ||': 'rain',
       'RA:02 FG:2 |FG RA |RA': 'rain',
       '-RA:02 FG:2 |FG RA |RA': 'rain',
       
       # Snow conditions
       '-SN:03 |SN |': 'snow',
       '-SN:03 BR:1 |SN |': 'snow',
       '|SN |': 'snow',
       '+SN:03 |SN s |': 'snow',
       '+SN:03 FZ:8 FG:2 |FG SN |': 'snow',
       '-SN:03 FZ:8 FG:2 |FG SN |': 'snow',
       'SN:03 FZ:8 FG:2 |FG SN |': 'snow',
       'SN:03 |SN s |s': 'snow',
       '-SN:03 FG:2 |FG SN |': 'snow',
       'SN:03 FG:2 |FG SN |': 'snow',
       
       # Other conditions
       'BR:1 ||': 'other',
       'HZ:7 |FU |HZ': 'other',
       'FG:2 |FG |': 'other',
       'HZ:7 ||HZ': 'other',
       'UP:09 ||': 'unknown',
       'UP:09 BR:1 ||': 'other'
   }

   # Map weather types
   weather_data['MappedWeather'] = weather_data['weather_type'].map(weather_mapping).fillna('other')

   def determine_weather_type(weather_series: pd.Series) -> str:
       """Determine daily weather type with priority: snow > rain > other."""
       if 'snow' in weather_series.values:
           return 'snow'
       elif 'rain' in weather_series.values:
           return 'rain'
       return 'other'

   # Aggregate to daily data
   daily_aggregated = weather_data.groupby('date').agg({
       'MappedWeather': determine_weather_type,
       'temperature': 'mean',
       'precipitation': 'mean',
       'wind_speed': 'mean'
   }).reset_index()

   # Rename columns to final format
   daily_aggregated.columns = [
       'date', 'daily weather type', 'daily temperature',
       'daily precipitation', 'daily windspeed'
   ]

   return daily_aggregated

In [ ]:
def load_and_clean_weather_data() -> tuple[pd.DataFrame, pd.DataFrame]:
   r"""
   Load and process weather data files into hourly and daily formats.
   
   Returns:
       tuple: Two DataFrames containing:
           - Hourly weather data concatenated from all input files
           - Daily weather data concatenated from all input files
           
   Notes:
       - Processes CSV files obtained from get_all_weather_csvs()
       - Cleans and standardizes both hourly and daily formats
       - Combines data from all years (2020-2024)
   """
   # Get list of weather CSV files
   weather_csv_files = get_all_weather_csvs(WEATHER_CSV_DIR)
   
   # Initialize lists to store processed DataFrames
   hourly_dataframes = []
   daily_dataframes = []

   # Process each CSV file
   for csv_file in weather_csv_files:
       hourly_dataframe = clean_month_weather_data_hourly(csv_file)
       daily_dataframe = clean_month_weather_data_daily(csv_file)
       hourly_dataframes.append(hourly_dataframe)
       daily_dataframes.append(daily_dataframe)
       
   # Combine all monthly data into single DataFrames
   hourly_data = pd.concat(hourly_dataframes)
   daily_data = pd.concat(daily_dataframes)
   
   return hourly_data, daily_data

In [ ]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

In [ ]:
hourly_weather_data.head()

In [ ]:
hourly_weather_data.info()

In [ ]:
hourly_weather_data.describe()

In [ ]:
daily_weather_data.head()

In [ ]:
daily_weather_data.info()

In [ ]:
daily_weather_data.describe()

## Part 2: Storing Cleaned Data

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather (
   date TIMESTAMP NOT NULL,
   hourly_weather_type VARCHAR(50),
   hourly_temperature FLOAT,
   hourly_precipitation FLOAT,
   hourly_windspeed FLOAT,
   hour INTEGER NOT NULL,
   weekday_num INTEGER NOT NULL,
   severe_weather FLOAT
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather (
   date DATE NOT NULL,
   daily_weather_type VARCHAR(50),
   avg_temperature FLOAT,
   avg_precipitation FLOAT, 
   avg_windspeed FLOAT
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips (
   pickup_datetime TIMESTAMP NOT NULL,
   dropoff_datetime TIMESTAMP NOT NULL,
   rate_code_id FLOAT,
   trip_distance FLOAT NOT NULL,
   extra FLOAT,
   mta_tax FLOAT,
   tip_amount FLOAT,
   tolls_amount FLOAT,
   improvement_surcharge FLOAT,
   total_amount FLOAT,
   congestion_surcharge FLOAT,
   airport_fee FLOAT,
   pickup_coords VARCHAR(50),
   dropoff_coords VARCHAR(50),
   weekday_num INTEGER NOT NULL,
   airport VARCHAR(50)
);
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips (
   hvfhs_license_num VARCHAR(50) NOT NULL,
   pickup_datetime TIMESTAMP NOT NULL,
   dropoff_datetime TIMESTAMP NOT NULL,
   trip_miles FLOAT,
   base_passenger_fare FLOAT,
   tolls FLOAT,
   sales_tax FLOAT,
   congestion_surcharge FLOAT,
   airport_fee FLOAT,
   driver_pay FLOAT,
   bcf FLOAT,
   pickup_coords VARCHAR(50),
   dropoff_coords VARCHAR(50),
   weekday_num INTEGER NOT NULL,
   total_amount FLOAT,
   tips FLOAT,
   airport VARCHAR(50)
);
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

In [ ]:
def write_dataframes_to_table(table_to_df_dict: dict[str, pd.DataFrame]) -> None:
   r"""
   Write multiple DataFrames to their corresponding database tables.

   Args:
       table_to_df_dict: Dictionary mapping table names to DataFrames

   Notes:
       - Creates tables if they don't exist using predefined schemas
       - Supported tables:
           - hourly_weather: Hourly weather measurements
           - daily_weather: Daily weather summaries  
           - taxi_trips: Yellow taxi trip records
           - uber_trips: Uber trip records
       - Standardizes column names before writing
       - Appends data if table already exists
   """
   # Create or verify tables exist
   with engine.connect() as conn:
       conn.execute(db.text(HOURLY_WEATHER_SCHEMA))
       conn.execute(db.text(DAILY_WEATHER_SCHEMA))
       conn.execute(db.text(TAXI_TRIPS_SCHEMA))
       conn.execute(db.text(UBER_TRIPS_SCHEMA))
       conn.commit()
       
       # Column mapping for each table
       column_mappings = {
           "hourly_weather": {
               'date': 'date',
               'hourly weather type': 'hourly_weather_type',
               'hourly temperature': 'hourly_temperature',
               'hourly precipitation': 'hourly_precipitation',
               'hourly windspeed': 'hourly_windspeed',
               'hour': 'hour',
               'weekday_num': 'weekday_num',
               'severe weather': 'severe_weather'
           },
           "daily_weather": {
               'date': 'date',
               'daily weather type': 'daily_weather_type',
               'daily temperature': 'avg_temperature',
               'daily precipitation': 'avg_precipitation',
               'daily windspeed': 'avg_windspeed'
           },
           "taxi_trips": {
               'pickup_datetime': 'pickup_datetime',
               'dropoff_datetime': 'dropoff_datetime',
               'RatecodeID': 'rate_code_id',
               'trip_distance': 'trip_distance',
               'extra': 'extra',
               'mta_tax': 'mta_tax',
               'tip_amount': 'tip_amount',
               'tolls_amount': 'tolls_amount',
               'improvement_surcharge': 'improvement_surcharge',
               'total_amount': 'total_amount',
               'congestion_surcharge': 'congestion_surcharge',
               'Airport_fee': 'airport_fee',
               'pickup_coords': 'pickup_coords',
               'dropoff_coords': 'dropoff_coords',
               'weekday_num': 'weekday_num',
               'airport': 'airport'
           },
           "uber_trips": {
               'hvfhs_license_num': 'hvfhs_license_num',
               'pickup_datetime': 'pickup_datetime',
               'dropoff_datetime': 'dropoff_datetime',
               'trip_miles': 'trip_miles',
               'base_passenger_fare': 'base_passenger_fare',
               'tolls': 'tolls',
               'sales_tax': 'sales_tax',
               'congestion_surcharge': 'congestion_surcharge',
               'airport_fee': 'airport_fee',
               'driver_pay': 'driver_pay',
               'bcf': 'bcf',
               'pickup_coords': 'pickup_coords',
               'dropoff_coords': 'dropoff_coords',
               'weekday_num': 'weekday_num',
               'total_amount': 'total_amount',
               'airport': 'airport',
               'tips': 'tips'
           }
       }
       
       # Write each DataFrame to its table
       for table_name, df in table_to_df_dict.items():
           # Standardize column names
           if table_name in column_mappings:
               df = df.rename(columns=column_mappings[table_name])
           
           # Write to database
           df.to_sql(
               name=table_name,
               con=engine,
               if_exists='append',
               index=False
           )
           print(f"Successfully wrote {len(df)} rows to table {table_name}")
            

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_weather_data,
    "daily_weather": daily_weather_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    df = pd.read_sql(query, engine)
    df.to_csv(outfile, index=False)

### Query 1

In [ ]:
QUERY_1 = """
WITH hourly_counts AS (
    SELECT 
        CAST(strftime('%H', pickup_datetime) AS INTEGER) as X,
        COUNT(*) as Y
    FROM taxi_trips
    WHERE 
        pickup_datetime >= '2020-01-01' 
        AND pickup_datetime < '2024-09-01'
    GROUP BY strftime('%H', pickup_datetime)
)
SELECT 
    X,
    Y,
    ROUND(CAST(Y AS FLOAT) * 100 / (SELECT SUM(Y) FROM hourly_counts), 2) as percentage
FROM hourly_counts
ORDER BY X;
"""

In [ ]:
conn = sqlite3.connect('project.db')
hourly_stats = pd.read_sql_query(QUERY_1, conn)
conn.close()
hourly_stats.to_csv("hourly_taxi_popularity.csv", index=False)

### Query 2

In [ ]:
QUERY_2 = """
WITH daily_counts AS (
    SELECT 
        weekday_num as X,
        COUNT(*) as Y
    FROM uber_trips
    WHERE 
        pickup_datetime >= '2020-01-01' 
        AND pickup_datetime < '2024-09-01'
    GROUP BY weekday_num
)
SELECT 
    X,
    Y,
    ROUND(CAST(Y AS FLOAT) * 100 / (SELECT SUM(Y) FROM daily_counts), 2) as percentage
FROM daily_counts
ORDER BY Y DESC;
"""

In [ ]:
conn = sqlite3.connect('project.db')
daily_stats = pd.read_sql_query(QUERY_2, conn)
conn.close()
daily_stats.to_csv("daily_uber_popularity.csv", index=False)

### Query 3

In [ ]:
QUERY_3 = """
WITH combined_trips AS (
    SELECT trip_distance as distance
    FROM taxi_trips
    WHERE 
        pickup_datetime >= '2024-01-01' 
        AND pickup_datetime < '2024-02-01'
    UNION ALL
    SELECT trip_miles as distance
    FROM uber_trips
    WHERE 
        pickup_datetime >= '2024-01-01' 
        AND pickup_datetime < '2024-02-01'
),
sorted_distances AS (
    SELECT 
        distance,
        (ROW_NUMBER() OVER (ORDER BY distance) - 1.0) / 
        (COUNT(*) OVER () - 1.0) * 100 as percentile
    FROM combined_trips
)
SELECT ROUND(distance, 2) as percentile_95
FROM sorted_distances
WHERE percentile >= 95
ORDER BY distance ASC
LIMIT 1;
"""

In [ ]:
conn = sqlite3.connect('project.db')
daily_stats = pd.read_sql_query(QUERY_3, conn)
conn.close()
daily_stats.to_csv("ride_distance_percentile.csv", index=False)

### Query 4

In [ ]:
QUERY_4 = """
WITH SnowDays AS (
    SELECT 
        date AS snow_date,
        avg_precipitation AS total_precipitation
    FROM daily_weather
    WHERE daily_weather_type = 'snow'
),
DailyRideCounts AS (
    SELECT 
        DATE(pickup_datetime) AS ride_date,
        COUNT(*) AS total_rides
    FROM (
        SELECT pickup_datetime FROM taxi_trips
        UNION ALL
        SELECT pickup_datetime FROM uber_trips
    )
    GROUP BY DATE(pickup_datetime)
)
SELECT 
    s.snow_date AS date,
    s.total_precipitation,
    COALESCE(d.total_rides, 0) AS total_rides
FROM SnowDays s
LEFT JOIN DailyRideCounts d
ON s.snow_date = d.ride_date
ORDER BY s.total_precipitation DESC
LIMIT 10;
"""

In [ ]:
conn = sqlite3.connect('project.db')
df = pd.read_sql_query(QUERY_4, conn)
df.to_csv("buiest_trip.csv", index=False)
conn.close()

### Query 5

In [ ]:
QUERY_5_FILENAME = "snowiest_days_rides.csv"
QUERY_5 = """
WITH SnowDays AS (
    SELECT 
        date AS snow_date,
        avg_precipitation AS total_precipitation
    FROM daily_weather
    WHERE daily_weather_type = 'snow'
),
DailyRideCounts AS (
    SELECT 
        DATE(pickup_datetime) AS ride_date,
        COUNT(*) AS total_rides
    FROM (
        SELECT pickup_datetime FROM taxi_trips
        UNION ALL
        SELECT pickup_datetime FROM uber_trips
    )
    GROUP BY DATE(pickup_datetime)
)
SELECT 
    s.snow_date AS date,
    s.total_precipitation,
    COALESCE(d.total_rides, 0) AS total_rides
FROM SnowDays s
LEFT JOIN DailyRideCounts d
ON s.snow_date = d.ride_date
ORDER BY s.total_precipitation DESC
LIMIT 10;
"""

In [ ]:
conn = sqlite3.connect('project.db')
df = pd.read_sql_query(QUERY_5, conn)
df.to_csv(QUERY_5_FILENAME, index=False)
conn.close()

### Query 6

In [ ]:
# Database connection configuration
conn = sqlite3.connect('project.db')
cursor = conn.cursor()

# SQL query to generate hourly data with weather and ride counts
query = """
WITH RECURSIVE GeneratedHours AS (
   SELECT datetime('2023-09-25 00:00:00') AS hour
   UNION ALL
   SELECT datetime(hour, '+1 hour') 
   FROM GeneratedHours
   WHERE hour < '2023-10-03 23:00:00'
),

HourlyWeatherData AS (
   SELECT 
       strftime('%Y-%m-%d %H:00:00', date || ' ' || hour || ':00:00') AS hour,
       AVG(hourly_precipitation) AS precipitation,
       AVG(hourly_windspeed) AS windspeed
   FROM hourly_weather
   WHERE date BETWEEN '2023-09-25' AND '2023-10-03'
   GROUP BY strftime('%Y-%m-%d %H:00:00', date || ' ' || hour || ':00:00')
),

HourlyRideCounts AS (
   SELECT 
       strftime('%Y-%m-%d %H:00:00', pickup_datetime) AS hour,
       COUNT(*) AS total_rides
   FROM (
       SELECT pickup_datetime FROM taxi_trips
       UNION ALL
       SELECT pickup_datetime FROM uber_trips
   )
   WHERE pickup_datetime BETWEEN '2023-09-25 00:00:00' AND '2023-10-03 23:59:59'
   GROUP BY strftime('%Y-%m-%d %H:00:00', pickup_datetime)
),

CombinedData AS (
   SELECT 
       g.hour AS datetime,
       COALESCE(r.total_rides, 0) AS total_rides,
       COALESCE(w.precipitation, 0.0) AS precipitation,
       COALESCE(w.windspeed, 0.0) AS windspeed
   FROM GeneratedHours g
   LEFT JOIN HourlyRideCounts r ON g.hour = r.hour
   LEFT JOIN HourlyWeatherData w ON g.hour = w.hour
)

SELECT 
   datetime,
   total_rides,
   precipitation,
   windspeed
FROM CombinedData
ORDER BY datetime ASC;
"""

try:
   # Execute query and fetch results
   cursor.execute(query)
   result_tuples = cursor.fetchall()

   # Display results
   for row in result_tuples:
       print(row)

finally:
   # Ensure database connection is properly closed
   cursor.close()
   conn.commit()
   conn.close()

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
def plot_hourly_taxi_distribution(dataframe: pd.DataFrame) -> None:
   r"""
   Create a two-panel visualization of hourly taxi ride distribution.

   Args:
       dataframe: DataFrame containing columns:
           - X: Hour of day (0-23)
           - Y: Number of rides
           - percentage: Percentage of total rides

   Notes:
       Creates two plots:
       - Top: Absolute number of rides by hour
       - Bottom: Percentage of total rides by hour
       Both include grid lines and value labels above each bar
   """
   # Set up figure with two subplots
   figure, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))

   # Plot absolute numbers
   ax1.bar(dataframe['X'], dataframe['Y'], color='orange', alpha=0.7)
   ax1.set_title('Hourly Distribution of Taxi Rides (2020-2024)', pad=20, size=14)
   ax1.set_xlabel('Hour of Day')
   ax1.set_ylabel('Number of Rides')
   ax1.grid(True, alpha=0.3)
   ax1.set_xticks(range(24))
   
   # Add value labels for absolute numbers
   for i, v in enumerate(dataframe['Y']):
       ax1.text(i, v, str(v), ha='center', va='bottom')
   
   # Plot percentages
   ax2.bar(dataframe['X'], dataframe['percentage'], color='green', alpha=0.7)
   ax2.set_title('Hourly Distribution of Taxi Rides (Percentage)', pad=20, size=14)
   ax2.set_xlabel('Hour of Day')
   ax2.set_ylabel('Percentage of Total Rides (%)')
   ax2.grid(True, alpha=0.3)
   ax2.set_xticks(range(24))
   
   # Add value labels for percentages
   for i, v in enumerate(dataframe['percentage']):
       ax2.text(i, v, f'{v:.2f}%', ha='center', va='bottom')
   
   plt.tight_layout()
   plt.show()


def get_hourly_taxi_data() -> pd.DataFrame:
   r"""
   Load hourly taxi ride data from CSV file.

   Returns:
       pd.DataFrame: Hourly taxi ride statistics
   """
   return pd.read_csv('hourly_taxi_popularity.csv')

In [ ]:
taxi_data_print = get_hourly_taxi_data()
plot_hourly_taxi_distribution(taxi_data_print)

### Visualization 2

In [ ]:
import numpy as np

# Use a separate assignment instead of inplace modification
uber_data = uber_data.rename(columns={'trip_miles': 'trip_distance'})

In [ ]:
combined_trips = pd.concat([
    taxi_data[['pickup_datetime', 'trip_distance']],
    uber_data[['pickup_datetime', 'trip_distance']]
])

start_date = '2020-01-01'
end_date = '2024-08-31'

combined_trips = combined_trips[
    (combined_trips['pickup_datetime'] >= start_date) &
    (combined_trips['pickup_datetime'] <= end_date)
]

combined_trips['month'] = combined_trips['pickup_datetime'].dt.month

# Group by month and calculate average distance and confidence interval
monthly_avg_distance = combined_trips.groupby('month')['trip_distance'].agg([
    'mean',
    'count',
    'std'
])

# Z-value for 90% confidence interval
z_value = 1.645

# Calculate standard error of mean and confidence intervals
monthly_avg_distance['sem'] = (
    monthly_avg_distance['std'] / np.sqrt(monthly_avg_distance['count'])
)
monthly_avg_distance['ci'] = z_value * monthly_avg_distance['sem']

# Reset index for plotting
monthly_avg_distance.reset_index(inplace=True)

# Convert data for plotting
months = monthly_avg_distance['month'].values.astype(float)
mean_values = monthly_avg_distance['mean'].values.astype(float)
lower_bound = (mean_values - monthly_avg_distance['ci'].values).astype(float)
upper_bound = (mean_values + monthly_avg_distance['ci'].values).astype(float)

# Create plot
plt.figure(figsize=(15, 12))
sns.lineplot(
    x=months,
    y=mean_values,
    label='Average Distance',
    color='blue'
)

plt.fill_between(
    months,
    lower_bound,
    upper_bound,
    color='b',
    alpha=0.2,
    label='90% Confidence Interval'
)

# Configure plot appearance
plt.xlabel('Month')
plt.ylabel('Average Distance (miles)')
plt.title(
    'Average Distance Traveled per Month (January 2020 - August 2024)\n'
    'Taxis and Ubers Combined'
)
plt.xticks(
    ticks=range(1, 13),
    labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
            'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
)
plt.legend()
plt.grid(visible=True)
plt.show()

### Visulization 3

In [ ]:
# Combine trip data from taxi and uber sources
combined_trips = pd.concat([
    taxi_data[['pickup_datetime', 'trip_distance', 'dropoff_coords']],
    uber_data[['pickup_datetime', 'trip_distance', 'dropoff_coords']]
])

# Define date range
start_date = '2020-01-01'
end_date = '2024-08-31'

# Filter trips within date range
combined_trips = combined_trips[
    (combined_trips['pickup_datetime'] >= start_date) &
    (combined_trips['pickup_datetime'] <= end_date)
]

# Define airport bounding boxes as constants
LGA_BOX_COORDS = (
    (40.763589, -73.891745),
    (40.778865, -73.854838)
)
JFK_BOX_COORDS = (
    (40.639263, -73.795642),
    (40.651376, -73.766264)
)
EWR_BOX_COORDS = (
    (40.686794, -74.194028),
    (40.699680, -74.165205)
)


def is_within_bbox(coord, bbox):
    """Check if coordinates fall within a bounding box."""
    lat, lon = coord
    (lat_min, lon_min), (lat_max, lon_max) = bbox
    return lat_min <= lat <= lat_max and lon_min <= lon <= lon_max


def determine_airport(row):
    """Determine which airport a dropoff location corresponds to."""
    coords = eval(row['dropoff_coords'].replace('POINT(', '').replace(')', ''))
    
    if is_within_bbox(coords, LGA_BOX_COORDS):
        return 'LGA'
    elif is_within_bbox(coords, JFK_BOX_COORDS):
        return 'JFK'
    elif is_within_bbox(coords, EWR_BOX_COORDS):
        return 'EWR'
    return None


# Add weekday and airport columns
combined_trips['weekday'] = combined_trips['pickup_datetime'].dt.day_name()
combined_trips['airport'] = combined_trips.apply(determine_airport, axis=1)

# Filter and analyze airport trips
airport_trips = combined_trips[combined_trips['airport'].isin(['LGA', 'JFK', 'EWR'])]
airport_popularity = (
    airport_trips.groupby(['airport', 'weekday'])
    .size()
    .reset_index(name='count')
)

# Create pivot table for visualization
weekday_order = [
    'Monday', 'Tuesday', 'Wednesday', 'Thursday',
    'Friday', 'Saturday', 'Sunday'
]
airport_popularity_pivot = airport_popularity.pivot(
    index='weekday',
    columns='airport',
    values='count'
).reindex(weekday_order)

# Create visualization
plt.figure(figsize=(12, 6))
airport_popularity_pivot.plot(kind='bar', figsize=(15, 12))
plt.xlabel('Day of the Week')
plt.ylabel('Number of Drop-offs')
plt.title('Most Popular Drop-off Days by Airport (January 2020 - August 2024)')
plt.xticks(rotation=45)
plt.legend(title='Airport')
plt.grid(visible=True)
plt.tight_layout()
plt.show()

### Visulization 4

### Visulization 5

In [ ]:
# Round pickup times to nearest hour
taxi_data['hour'] = taxi_data['pickup_datetime'].dt.round('H')
uber_data['hour'] = uber_data['pickup_datetime'].dt.round('H')
hourly_weather_data['hour'] = hourly_weather_data['date'].dt.round('H')

# Merge weather data with trip data
taxi_merged = pd.merge(
   taxi_data,
   hourly_weather_data[['hour', 'hourly precipitation']],
   on='hour',
   how='left'
)
uber_merged = pd.merge(
   uber_data,
   hourly_weather_data[['hour', 'hourly precipitation']],
   on='hour',
   how='left'
)

# Filter out invalid data points
taxi_merged = taxi_merged[
   (taxi_merged['trip_distance'] > 0) &
   (taxi_merged['tip_amount'] > 0)
]
uber_merged = uber_merged[
   (uber_merged['trip_distance'] > 0) &
   (uber_merged['tips'] > 0)
]

# Remove outliers above 99th percentile
taxi_merged = taxi_merged[
   (taxi_merged['trip_distance'] < np.percentile(taxi_merged['trip_distance'], 99)) &
   (taxi_merged['tip_amount'] < np.percentile(taxi_merged['tip_amount'], 99))
]
uber_merged = uber_merged[
   (uber_merged['trip_distance'] < np.percentile(uber_merged['trip_distance'], 99)) &
   (uber_merged['tips'] < np.percentile(uber_merged['tips'], 99))
]

# Create visualization grid
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot taxi tips vs distance
axes[0, 0].scatter(
   taxi_merged['trip_distance'],
   taxi_merged['tip_amount'],
   alpha=0.5,
   color='blue'
)
axes[0, 0].set_xlabel('Distance (miles)')
axes[0, 0].set_ylabel('Tip Amount ($)')
axes[0, 0].set_title('Yellow Taxi Tips vs. Distance')

# Plot Uber tips vs distance
axes[0, 1].scatter(
   uber_merged['trip_distance'],
   uber_merged['tips'],
   alpha=0.5,
   color='red'
)
axes[0, 1].set_xlabel('Distance (miles)')
axes[0, 1].set_ylabel('Tip Amount ($)')
axes[0, 1].set_title('Uber Tips vs. Distance')

# Plot taxi tips vs precipitation
axes[1, 0].scatter(
   taxi_merged['hourly precipitation'],
   taxi_merged['tip_amount'],
   alpha=0.5,
   color='green'
)
axes[1, 0].set_xlabel('Hourly Precipitation (inches)')
axes[1, 0].set_ylabel('Tip Amount ($)')
axes[1, 0].set_title('Yellow Taxi Tips vs. Precipitation')

# Plot Uber tips vs precipitation
axes[1, 1].scatter(
   uber_merged['hourly precipitation'],
   uber_merged['tips'],
   alpha=0.5,
   color='purple'
)
axes[1, 1].set_xlabel('Hourly Precipitation (inches)')
axes[1, 1].set_ylabel('Tip Amount ($)')
axes[1, 1].set_title('Uber Tips vs. Precipitation')

plt.tight_layout()
plt.show()

### Visulization 6

In [ ]:
# Initialize map centered on NYC
m = folium.Map(
   location=[40.7128, -74.0060],
   zoom_start=11
)

# Process taxi pickup coordinates
taxi_coords = [
   [
       float(coord.split(',')[0]),
       float(coord.split(',')[1])
   ] 
   for coord in taxi_data['pickup_coords']
]

# Add taxi heatmap layer
taxi_gradient = {
   0.4: 'yellow',
   0.65: 'orange',
   1: 'red'
}
HeatMap(
   taxi_coords,
   radius=15,
   gradient=taxi_gradient
).add_to(m)

# Process Uber pickup coordinates  
uber_coords = [
   [
       float(coord.split(',')[0]), 
       float(coord.split(',')[1])
   ]
   for coord in uber_data['pickup_coords']
]

# Add Uber heatmap layer
uber_gradient = {
   0.4: 'blue',
   0.65: 'purple',
   1: 'red'
}
HeatMap(
   uber_coords,
   radius=15,
   gradient=uber_gradient
).add_to(m)

# Save final map
m.save('nyc_rides_heatmap_2020.html')

In [ ]:
uber_data